# Point cloud registration with Easy Open3D

In [1]:
from easy_o3d import utils
from easy_o3d.registration import IterativeClosestPoint, FastGlobalRegistration, RANSAC, ICPTypes, KernelTypes
from easy_o3d import set_logger_level

import logging
import copy
import open3d as o3d
import numpy as np
import json
import time
import itertools
import joblib
import tqdm

set_logger_level(logging.WARNING)

## Load and visualize point clouds

In [ ]:
# Load ground truth pose
with open("tests/test_data/ground_truth_pose.json") as f:
    gt_pose_data = json.load(f)
rotation = gt_pose_data["rotation_quaternion"]
translation = gt_pose_data["translation_xyz"]
gt_pose = utils.get_transformation_matrix_from_quaternion(rotation_wxyz=rotation, translation_xyz=translation)

source_path = "tests/test_data/suzanne.ply"
source = utils.eval_data(data=source_path, number_of_points=10000)

target_path = "tests/test_data/suzanne_on_chair.ply"
target = utils.eval_data(data=target_path, number_of_points=100000)

# utils.draw_geometries(geometries=[source, target])

In [ ]:
source = utils.process_point_cloud(point_cloud=source,
                                   downsample=utils.DownsampleTypes.VOXEL,
                                   downsample_factor=0.01)

source = utils.process_point_cloud(point_cloud=source,
                                   estimate_normals=True,
                                   search_param_knn=30,
                                   search_param_radius=0.02,
                                   recalculate_normals=True)

_, source_feature = utils.process_point_cloud(point_cloud=source,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

target = utils.process_point_cloud(point_cloud=target,
                                   downsample=utils.DownsampleTypes.VOXEL,
                                   downsample_factor=0.01)

target = utils.process_point_cloud(point_cloud=target,
                                   estimate_normals=True,
                                   search_param_knn=30,
                                   search_param_radius=0.02,
                                   recalculate_normals=True)

_, target_feature = utils.process_point_cloud(point_cloud=target,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

## RANSAC

In [ ]:
ransac = RANSAC()
ransac_result = ransac.run(source=source,
                           target=target,
                           source_feature=source_feature,
                           target_feature=target_feature,
                           draw=True,
                           overwrite_colors=True)

print(np.linalg.norm(gt_pose - ransac_result.transformation))

In [ ]:
ransac = RANSAC()
ransac_result = ransac.run_multi_scale(source=source,
                                       target=target,
                                       source_scales = [0.02, 0.01, 0.005],
                                       iterations=[100000, 50000, 20000],
                                       radius_multiplier=[2, 3])

print(np.linalg.norm(gt_pose - ransac_result.transformation))

## Fast Global Registration

In [ ]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run(source=source,
                     target=target,
                     source_feature=source_feature,
                     target_feature=target_feature,
                     draw=True,
                     overwrite_colors=True)

print(np.linalg.norm(gt_pose - fgr_result.transformation))

In [ ]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run_multi_scale(source=source,
                                    target=target,
                                    source_scales = [0.02, 0.01, 0.005])

print(np.linalg.norm(gt_pose - fgr_result.transformation))

## Iterative Closest Point

In [ ]:
icp = IterativeClosestPoint(estimation_method=ICPTypes.PLANE)
result = icp.run(source=source,
                 target=target,
                 init=ransac_result.transformation,
                 max_iteration=100,
                 draw=True,
                 overwrite_colors=True)

print(np.linalg.norm(gt_pose - result.transformation))

In [5]:
source_path = "tests/test_data/suzanne.ply"
target_path = "tests/test_data/suzanne_on_chair.ply"
source = utils.eval_data(source_path, number_of_points=10000)
target = utils.eval_data(target_path, number_of_points=10000)
source_list = [source] * 3
target_list = [target] * 100

icp = IterativeClosestPoint(max_iteration=100000, relative_rmse=1e-30, relative_fitness=1e-30)

start = time.time()
icp.run_many(source_list=source_list,
             target_list=target_list,
             init_list=[[0, 0, 0.5]] * 300,
             number_of_points=10000)
print(f"Took {time.time() - start} seconds.")

Took 9.997875452041626 seconds.


## Load camera parameters from BlenderProc BopWriter

In [ ]:
scene_id = 2
path_to_scene_camera_json = "tests/test_data/bop_data/obj_of_interest/train_pbr/000000/scene_camera.json"
path_to_camera_json = "tests/test_data/bop_data/obj_of_interest/camera.json"
output_path = "tests/test_data"

camera_parameters = utils.get_camera_parameters_from_blenderproc_bopwriter(scene_id,
                                                                           path_to_scene_camera_json,
                                                                           path_to_camera_json,
                                                                           output_path)

## Load RGB-D image

In [ ]:
camera_parameters = o3d.io.read_pinhole_camera_parameters("tests/test_data/camera_parameters.json")
color = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/rgb/000002.png"
depth = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/depth/000002.png"

target_rgbd = utils.convert_rgbd_image_to_point_cloud(rgbd_image_or_path=[color, depth],
                                                      camera_intrinsic=camera_parameters.intrinsic,
                                                      camera_extrinsic=camera_parameters.extrinsic,
                                                      depth_scale=1000.0,
                                                      depth_trunc=2.0)

target_rgbd = utils.process_point_cloud(point_cloud=target_rgbd,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.01,
                                        remove_outlier=utils.OutlierTypes.RADIUS,
                                        outlier_std_ratio=1.0,
                                        search_param_knn=20,
                                        search_param_radius=0.2)

target_rgbd = utils.process_point_cloud(point_cloud=target_rgbd,
                                        estimate_normals=True,
                                        search_param_knn=30,
                                        search_param_radius=0.02)

_, target_rgbd_feature = utils.process_point_cloud(point_cloud=target_rgbd,
                                                   compute_feature=True,
                                                   search_param_knn=100,
                                                   search_param_radius=0.05)

utils.draw_geometries(geometries=[target_rgbd])

## RANSAC

In [ ]:
ransac = RANSAC()
ransac_result = ransac.run(source=source,
                           target=target_rgbd,
                           source_feature=source_feature,
                           target_feature=target_rgbd_feature,
                           draw=True,
                           overwrite_colors=True)

print(np.linalg.norm(gt_pose - ransac_result.transformation))

## Fast Global Registration

In [ ]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run(source=source,
                     target=target_rgbd,
                     source_feature=source_feature,
                     target_feature=target_rgbd_feature,
                     draw=True,
                     overwrite_colors=True)

print(np.linalg.norm(gt_pose - fgr_result.transformation))

## Iterative Closest Point

In [ ]:
icp = IterativeClosestPoint()
result = icp.run(source=source,
                 target=target_rgbd,
                 estimation_method=ICPTypes.PLANE,
                 init=ransac_result.transformation,
                 crop_target_around_source=True,
                 crop_scale=1.5,
                 max_iteration=100,
                 max_correspondence_distance=0.004,
                 draw=True,
                 overwrite_colors=False)

print(np.linalg.norm(gt_pose - result.transformation))